## 1. Importing Libraries

In [42]:
import pickle

import mlflow

import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBRegressor

import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import(
      r2_score,
      mean_squared_error,
      root_mean_squared_error
)
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.base import(
      BaseEstimator,
      TransformerMixin
)
from sklearn.pipeline import(
      Pipeline,
      FeatureUnion
)
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import(
      OneHotEncoder,
      OrdinalEncoder,
      StandardScaler,
      MinMaxScaler,
      PowerTransformer,
      FunctionTransformer
)

from feature_engine.outliers import Winsorizer
from feature_engine.encoding import(
      RareLabelEncoder,
      MeanEncoder,
      CountFrequencyEncoder
)
from feature_engine.datetime import DatetimeFeatures
from feature_engine.selection import SelectBySingleFeaturePerformance

from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns',None)
sklearn.set_config(transform_output = "pandas")

## 2. Reading the datasets

In [2]:
PROJECT_DIR = Path(r'F:\Rishabh\Flight-Price-Prediction')
DATA_DIR = "data"

In [3]:
def read_data(name):
    file_name = f"{name}/{name}.csv"
    file_path = PROJECT_DIR/DATA_DIR/file_name

    return pd.read_csv(file_path)

In [4]:
train = read_data("train")
test = read_data("test")
val = read_data("val")

## 3. Preprocessing Operations

In [5]:
#airline

air_transformer = Pipeline(
    steps = [
        ("imputer",SimpleImputer(strategy = "most_frequent")),
        ("grouper",RareLabelEncoder(tol = 0.1, replace_with = "Other",n_categories = 2)),
        ("encoder",OneHotEncoder(sparse_output = False, handle_unknown = "ignore"))
    ]
)

# date of journey

features_to_extract = ["month","week","day_of_week","day_of_year"]

doj_transformer = Pipeline(steps = [
    ("dt",DatetimeFeatures(features_to_extract = features_to_extract, yearfirst = True, format = "mixed")),
    ("scaler",MinMaxScaler())
])

# source and destination

location_pipe = Pipeline(steps = [
    ("grouper",RareLabelEncoder(tol = 0.1,replace_with = "Other",n_categories = 2)),
    ("encoder",MeanEncoder()),
    ("scaler",PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ["Delhi","Kolkata","Mumbai","New Delhi"]
    return (
        X
        .assign(
            **{
                f"{col}_isnorth": X.loc[:,col].isin(north_cities).astype('int')
                for col in columns
            }
        )
        .drop(columns = columns)
    )

location_transformer = FeatureUnion(transformer_list = [
    ("pipe1",location_pipe),
    ("pipe2",FunctionTransformer(func = is_north))
])

# dep time and arrival time

def part_of_the_day(X,morning = 4,afternoon = 12, evening =16,night = 20):
    columns = X.columns.to_list()
    X_temp = X.assign(**{
        col: pd.to_datetime(X.loc[:,col]).dt.hour
        for col in columns
    })

    return(
        X_temp.assign(**{
                f"{col}_part_of_day" : np.select(
                    [X_temp.loc[:,col].between(morning,afternoon,inclusive = "left"),
                     X_temp.loc[:,col].between(afternoon,evening,inclusive = "left"),
                     X_temp.loc[:,col].between(evening,night,inclusive = "left")],
                    ["morning","afternoon","evening"],
                    default = "night"
            )

                for col in columns
        })
        .drop(columns = columns)
    )

time_pipe = Pipeline(steps= [
    ("dt",DatetimeFeatures(features_to_extract = ["hour","minute"])),
])

time_pipe2 = Pipeline(steps =[
    ("part",FunctionTransformer(part_of_the_day)),
    ("encoder",CountFrequencyEncoder()),
    ("scaler",MinMaxScaler())
])

time_transformer = FeatureUnion(transformer_list = [
    ("part1",time_pipe),
    ("part2",time_pipe2)
])

# duration

class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self,variables = None, percentiles = [0.25,0.5,0.75], gamma = 0.1):
        self.variables = variables
        self.percentiles = percentiles
        self.gamma = gamma

    def fit(self,X,y = None):
        if not self.variables:
            self.variables = X.select_dtypes(include = "number").columns.to_list()

        self.reference_values_ = {
            col: (
                X
                .loc[:,col]
                .quantile(self.percentiles)
                .values
                .reshape(-1,1)
            )
            for col in self.variables
        }

        return self

    def transform(self,X):
         objects = []
         for col in self.variables:
             columns = [f"{col}_rbf_{int(percentile*100)}" for percentile in self.percentiles]
             obj = pd.DataFrame(
                 data = rbf_kernel(X.loc[:,[col]], Y = self.reference_values_[col], gamma = self.gamma),
                 columns = columns
             )
             objects.append(obj)
         return pd.concat(objects,axis=1)

rbf = RBFPercentileSimilarity()

def duration_category(X,short = 180,med =400):
    return (
        X.assign(
        duration_cat = np.select(
            [X.duration.lt(short),
             X.duration.between(short,med,inclusive = "left")],
            ["short","medium"],
            default = "long"
        )
    )
    .drop(columns = "duration")
   )

def is_over(X,value = 1000):
    return(
        X.assign(**{
            f"duration_over_{value}": X.duration.gt(1000).astype(int)
        })
        .drop(columns = "duration")
    )

duration_pipe1 = Pipeline(steps = [
    ("rbf",RBFPercentileSimilarity()),
    ("scaler",PowerTransformer())
])

duration_pipe2 = Pipeline(steps=[
	("cat", FunctionTransformer(func=duration_category)),
	("encoder", OrdinalEncoder(categories=[["short", "medium", "long"]]))
])

duration_union = FeatureUnion(transformer_list=[
	("part1", duration_pipe1),
	("part2", duration_pipe2),
	("part3", FunctionTransformer(func=is_over)),
	("part4", StandardScaler())
])

duration_transformer = Pipeline(steps=[
	("outliers", Winsorizer(capping_method="iqr", fold=1.5)),
	("imputer", SimpleImputer(strategy="median")),
	("union", duration_union)
])

# total stops

def is_direct(X):
	return X.assign(is_direct_flight=X.total_stops.eq(0).astype(int))


total_stops_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("", FunctionTransformer(func=is_direct))
])

# additional info

info_pipe1 = Pipeline(steps=[
	("group", RareLabelEncoder(tol=0.1, n_categories=2, replace_with="Other")),
	("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

def have_info(X):
	return X.assign(additional_info=X.additional_info.ne("No Info").astype(int))

info_union = FeatureUnion(transformer_list=[
	("part1", info_pipe1),
	("part2", FunctionTransformer(func=have_info))
])

info_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
	("union", info_union)
])

# column transformer

column_transformer = ColumnTransformer(transformers = [
    ("air",air_transformer,["airline"]),
    ("doj",doj_transformer,["date_of_journey"]),
    ("location",location_transformer,["source","destination"]),
    ("time",time_transformer,["arrival_time","dep_time"]),
    ("dur", duration_transformer, ["duration"]),
	("stops", total_stops_transformer, ["total_stops"]),
	("info", info_transformer, ["additional_info"])
], remainder = "passthrough")

# feature selection

estimator = RandomForestRegressor()

selector = SelectBySingleFeaturePerformance(
	estimator=estimator,
	scoring="r2",
	threshold=0.1
) 

# final preprocessor

preprocessor = Pipeline(steps=[
	("ct", column_transformer),
	("selector", selector)
])

In [6]:
preprocessor.fit(
    train.drop(columns = ["price"]),
    train.price.copy()
)

,steps,"[('ct', ...), ('selector', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('air', ...), ('doj', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [7]:
preprocessor_params = preprocessor.get_params()

## 4. Model Pipeline

In [43]:
XGBRegressor().get_params()

{'objective': 'reg:squarederror',
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'device': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'feature_weights': None,
 'gamma': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'multi_strategy': None,
 'n_estimators': None,
 'n_jobs': None,
 'num_parallel_tree': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [70]:
model_params = {
     'objective': 'reg:squarederror',
     'base_score': None,
     'booster': None,
     'callbacks': None,
     'colsample_bylevel': None,
     'colsample_bynode': None,
     'colsample_bytree': None,
     'device': None,
     'early_stopping_rounds': None,
     'enable_categorical': False,
     'eval_metric': None,
     'feature_types': None,
     'feature_weights': None,
     'gamma': None,
     'grow_policy': None,
     'importance_type': None,
     'interaction_constraints': None,
     'learning_rate': 0.01,
     'max_bin': None,
     'max_cat_threshold': None,
     'max_cat_to_onehot': None,
     'max_delta_step': None,
     'max_depth': 12,
     'max_leaves': None,
     'min_child_weight': None,
     'missing': np.nan,
     'monotone_constraints': None,
     'multi_strategy': None,
     'n_estimators': 200,
     'n_jobs': -1,
     'num_parallel_tree': None,
     'random_state': None,
     'reg_alpha': None,
     'reg_lambda': None,
     'sampling_method': None,
     'scale_pos_weight': None,
     'subsample': 0.7,
     'tree_method': None,
     'validate_parameters': None,
     'verbosity': None
}

In [71]:
model_pipe = Pipeline(steps = [
    ('preprocessor',preprocessor),
    ('model',XGBRegressor(**model_params))
])

model_pipe

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('ct', ...), ('selector', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('air', ...), ('doj', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3


In [72]:
model_pipe.fit(
    train.drop(columns = "price"),
    train.price.copy()
)

,steps,"[('preprocessor', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,steps,"[('ct', ...), ('selector', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('air', ...), ('doj', ...), ...]"
,remainder,'passthrough'
,sparse_threshold,0.3


## 5. Model Evaluation

In [73]:
y_pred = model_pipe.predict(test.drop(columns = "price"))

r2 = r2_score(test.price, y_pred)
mse = mean_squared_error(test.price, y_pred)

In [74]:

metrics = {
    'r2_score': r2,
    'mean_squared_error': mse
}

metrics

{'r2_score': 0.7853313684463501, 'mean_squared_error': 4537884.0}

## 6. MLFlow Tracking

In [75]:
model_pipe.get_params()

{'memory': None,
 'steps': [('preprocessor', Pipeline(steps=[('ct',
                    ColumnTransformer(remainder='passthrough',
                                      transformers=[('air',
                                                     Pipeline(steps=[('imputer',
                                                                      SimpleImputer(strategy='most_frequent')),
                                                                     ('grouper',
                                                                      RareLabelEncoder(n_categories=2,
                                                                                       replace_with='Other',
                                                                                       tol=0.1)),
                                                                     ('encoder',
                                                                      OneHotEncoder(handle_unknown='ignore',
                                   

In [20]:
mlflow.is_tracking_uri_set()

True

In [80]:
mlflow.set_tracking_uri('http://127.0.0.1:5000')

mlflow.set_experiment("Flight Price Prediction Experiment")

with mlflow.start_run(run_name = "xgboost fifth run") as run:

    signature = mlflow.models.infer_signature(model_input = train.drop(columns = "price"), model_output = model_pipe.predict(test.drop(columns = "price")))
    
    mlflow.log_params(model_pipe.get_params())

    mlflow.log_metrics(metrics)

    mlflow.sklearn.log_model(sk_model = model_pipe,  name = "model", signature = signature)

🏃 View run xgboost fifth run at: http://127.0.0.1:5000/#/experiments/791845741988631666/runs/48223302862248c4aef1632323b1d28e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/791845741988631666
